

# Pipeline complet pour l'évaluation et l'amélioration d'un modèle ML : analyse du dataset d'OpenFoodFact

Ce notebook présente une approche complète pour entraîner, évaluer et améliorer un modèle de machine learning. Il comprend plusieurs étapes, telles que la préparation des données, l'implémentation des modèles, l'analyse des erreurs et la mise en production du modèle.


## Chargement des données 

### A partir du format csv

In [2]:
import pandas as pd

In [3]:
path = "https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv.gz"
df = pd.read_csv(path, nrows=100, sep='\t',encoding="utf-8")

In [4]:
df

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,last_modified_by,last_updated_t,last_updated_datetime,...,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g,sulphate_100g,nitrate_100g,acidity_100g
0,54,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1582569031,2020-02-24T18:30:31Z,1733085204,2024-12-01T20:33:24Z,NaN,1736276870,2025-01-07T19:07:50Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,63,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1673620307,2023-01-13T14:31:47Z,1732913331,2024-11-29T20:48:51Z,insectproductadd,1738686803,2025-02-04T16:33:23Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,114,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1580066482,2020-01-26T19:21:22Z,1737247862,2025-01-19T00:51:02Z,smoothie-app,1738687801,2025-02-04T16:50:01Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,http://world-en.openfoodfacts.org/product/0000...,inf,1634745456,2021-10-20T15:57:36Z,1738676541,2025-02-04T13:42:21Z,waistline-app,1738684455,2025-02-04T15:54:15Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,105,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1572117743,2019-10-26T19:22:23Z,1738073570,2025-01-28T14:12:50Z,NaN,1738685184,2025-02-04T16:06:24Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,91,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1536994879,2018-09-15T07:01:19Z,1729528185,2024-10-21T16:29:45Z,roboto-app,1738681915,2025-02-04T15:11:55Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,92,http://world-en.openfoodfacts.org/product/0000...,product-scan-com,1638580447,2021-12-04T01:14:07Z,1728651371,2024-10-11T12:56:11Z,fix-code-bot,1738682899,2025-02-04T15:28:19Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,93,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1537041112,2018-09-15T19:51:52Z,1728236141,2024-10-06T17:35:41Z,macrofactor,1738687164,2025-02-04T16:39:24Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,94,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1572449868,2019-10-30T15:37:48Z,1728034902,2024-10-04T09:41:42Z,fix-code-bot,1737510101,2025-01-22T01:41:41Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 206 columns):
 #    Column                                                 Dtype  
---   ------                                                 -----  
 0    code                                                   int64  
 1    url                                                    object 
 2    creator                                                object 
 3    created_t                                              int64  
 4    created_datetime                                       object 
 5    last_modified_t                                        int64  
 6    last_modified_datetime                                 object 
 7    last_modified_by                                       object 
 8    last_updated_t                                         int64  
 9    last_updated_datetime                                  object 
 10   product_name                                           object

### A partir du format parquet 

parquet est un format optimisé pour la maniupulation de gros data set

on peut charger le data set sous ce format, à partir de [son emplacement sur HuggingFace](https://huggingface.co/datasets/openfoodfacts/product-database) (attention il faudra installer les librairies suivantes pour cela) 

Si ca n'est pas déja fait, télécharger les librairies nécessaires : 
```
pip install huggingface-hub
pip fastparquet

```

In [ ]:
# Login using e.g. `huggingface-cli login` to access this dataset
splits = {'food': 'food.parquet', 'beauty': 'beauty.parquet'}
df = pd.read_parquet("hf://datasets/openfoodfacts/product-database/" + splits["food"])

### Autres méthodes

Pour des détails complets sur les différentes options pour charger les données, consultez la [page dédiée du projet](https://world.openfoodfacts.org/data)